# State Monad

## Outline

* Incentive for the State monad

* The State monad
  - Definition of the State monad

  - MonadState type class
  
  - Simple example
  
* State monad examples
  - State monad with IO actions

In this lesson, we will learn about the State monad type and how you can use it.

## Incentive for State monad

We talked in the previous two lectures how you can use the Reader and Writer Monad if you have to read from a environment variable or write to it.

If you need to perform both operations you can use the State Monad. Let's first look at an example where we perform read and write operations without the State monad.

We will create the Tic-Tac-Toe game [(1)](https://en.wikipedia.org/wiki/Tic-tac-toe) that will use the **System.Random** module to randomly pick the X and O choices.

In [ ]:
import System.Random (mkStdGen, Random(randomR), StdGen)

data Player = XPlayer | OPlayer deriving Eq
data Choice = Empty | X | O deriving Eq

data GameState = GameState
  { currentBoard :: [Choice]
  , currentPlayer :: Player
  , generator :: StdGen
  }

main :: IO ()
main = do
    putStrLn "Game results:"
    let gen = mkStdGen 1
        initState = GameState
                      [Empty | boardInd <- [1..9]]
                      XPlayer
                      gen
    playGame initState
    
playGame :: GameState -> IO ()
playGame gs = do
    let freeFields = getFreeFields gs
    if length freeFields /= 0
    then do
        let player = currentPlayer gs
            board = currentBoard gs
            gen = generator gs
            (choiceInd, gen') = randomR (0, length freeFields - 1) gen
            choice = (freeFields !! choiceInd) + 1
            newGameState = GameState
                            (if player == XPlayer
                            then [if ind /= choice then board !! (ind-1) else X | ind <- [1..9]]
                            else [if ind /= choice then board !! (ind-1) else O | ind <- [1..9]])
                            (nextPlayer player)
                            gen'
        playGame newGameState
    else do
        printBoard gs

getFreeFields :: GameState -> [Int]
getFreeFields gs = [ind | ind <- [0..8], board !! ind == Empty]
    where board = currentBoard gs

nextPlayer :: Player -> Player
nextPlayer XPlayer = OPlayer
nextPlayer OPlayer = XPlayer

printBoard :: GameState -> IO ()
printBoard gs = do
    let board = currentBoard gs
    let stateToString st = case st of
                             Empty -> "-"
                             X -> "X"
                             O -> "O"
        printInd ind = stateToString $ board !! ind
    mapPutStr [printInd 0,"|", printInd 1,"|", printInd 2, "\n"]
    putStrLn "-----"
    mapPutStr [printInd 3,"|", printInd 4,"|", printInd 5, "\n"]
    putStrLn "-----"
    mapPutStr [printInd 6,"|", printInd 7,"|", printInd 8, "\n"]

mapPutStr :: [String] -> IO ()
mapPutStr [] = return ()
mapPutStr [x] = putStr x
mapPutStr (x:xs) = putStr x >> mapPutStr xs

main

In the code above we define the **GameState** variable that holds data for the current state of the game. In the **main** function we created the initial state.

In the **playGame** function we use the **GameState** variable to create a new move and update the variable before we recursivly call this function again or print the game results.

This is a simple game but if it would become more complex you would need to pass around a lot of data. This is where the **State monad** can help you out. 

## The State monad

### Definition of the State monad

The Haskell2010 definition of the `State` type is defined in terms of the `StateT` monad transformer:
```haskell
type State s = StateT s Identity
```
We will learn about Monad transformers in lesson 25.

You can also define the State monad without using a monad transformer that works equally as the one above. We will chose this aproach in this lesson. 

Let's first look at the definition of the **State** type:
```haskell
newtype State s a = State { runState :: s -> (a, s) }
```

We see that the `State` data constructor holds a function that can be accessed with the name `runState`.

It takes in a state and returns a tuple that contains a variable of type `a` and another state variable.

We can now create a Monad instance for `State s` and not just `State`.

This means the type of our state will remain the same as we compose our function with `(>>=)`.
```haskell
(>>=) :: State s a        ->
         (a -> State s b) ->
         State s b
```

The monad instance for `State s` would be defined as:
```haskell
instance Monad (State s) where
    return a = State $ \st -> (a, st)
    stMonad >>= f = State $ \st1 -> 
        let (a, st2) = runState stMonad st1
        in runState (f a) st2
```

You could then define the Functor and Applicative instances with the help of the `ap` and `liftM` function from the **Control.Monad** module.

Here we will define the Functor and Applicative instances without the help of these monadic functions.
```haskell
instance Functor (State s) where
fmap f stFunctor = State $ \st1 ->
    let (a, st2) = runState stFunctor st1
    in (f a, st2)

instance Applicative (State s) where
pure a = State $ \st -> (a, st)
f <*> g = State $ \st1 ->
    let (h, st2) = runState f st1
        (a, st3) = runState g st2
    in (h a, st3)
```

### Helper functions

The **Control.Monad.Trans.State** module defines some helper functions for the State moand. 

You have the option to import this module as a lazy or a strict module by prepending **.Lazy** or **.Strict**.

Commonly used helper functions are `get` and `put` that are used inside a State monad. 
```haskell
get :: (Monad m) => State s s
get = state $ \s -> (s, s)

put :: (Monad m) => s -> State s ()
put s = state $ \_ -> ((), s)
```

The `get` function gets the state variable of the State monad. And the `put` function sets the state variable of the State monad. 

Then there are the `evalState` and `execState` helper functions.
```haskell
evalState :: State s a -> s -> a
evalState stateAction initialState =
    fst $ runState stateAction initialState

execState :: State s a -> s -> s
execState stateAction initialState =
    snd $ runState stateAction initialState
```

The difference to the functions `get` and `put` is that they are used outside of a state monad.

The `evalState` function returns the result variable of the State monad. The `execState` function returns the state variable of the State monad.  

Two other helper functions that this module also contains are the `modify` and `gets` functions.
```haskell
modify :: MonadState s m => (s -> s) -> m ()
modify f = state (\st -> ((), f st))

gets :: MonadState s m => (s -> a) -> m a
gets f = do
    st <- get
    return (f st)
```

The `modify` function takes in a function and updates the state monad such that is applies the function to the state variable.

The `gets` function takes in a function, applies it to the state variable and writes the result in the type `a` variable of the state monad.

### MonadState type class

The `MonadState` type class is defined in the **Control.Monad.State.Class** module which is part of the **mtl** package.

This type class defines the `get` and `put` functions and the module defines also the previously mentioned helper functions. 

We look at the definition of the `MonadState` type class to learn a new way how to define type classes and class instances.

The `MonadState` type class definition is:
```haskell
class Monad m => MonadState s m | m -> s where
    get :: m s
    get = state (\s -> (s, s))

    put :: s -> m ()
    put s = state (\_ -> ((), s))

    state :: (s -> (a, s)) -> m a
    state f = do
      s <- get
      let ~(a, s') = f s
      put s'
      return a
    {-# MINIMAL state | get, put #-}
```

The minimal complete definition are the `get` and `put` functions which can get or set the state variable of the state moand.

You can also alternativly define the `state` function that takes in a function and returns a state monad. 

To be able to declare a type class with two types you need to use the *MultiParamTypeClasses* language pragma.

If the type `m` which in reality for us will be `(State s)`, is encapsulating the type `s` we can state this dependency with the `| m -> s` statement.

To be able to use such dependency statements in type class declarations we need to include the *FunctionalDependencies* language pragma.

For our State monad we could write an instance of the `MonadState` type class as:
```haskell
instance MonadState s (State s) where
    put :: s -> State s ()
    put st = State $ \_ -> ((), st)

    get :: State s s
    get = State $ \st -> (st, st)
```

To be able to do this we would also need to include the language pragmas *FlexibleInstances* and *InstanceSigs*.

The first let's you work with concrete types when creating an type class instances. For example if you want to create an instance for `Maybe Int` instead of `Maybe a` you can do this.

The second enables to also write our type signatures as part of typeclass instance declarations. 

### Simple example

In practical Haskell code we do not define functions, types and type classes from the previous chapters on our own. 

We can import them from the **Control.Monad.State** module.

Let's write an example of a State monad that helps us compute the first n fibonacci numbers.

In [ ]:
import Control.Monad.State (MonadState(put, get), runState, State)

type FibNums = [Int]

printFibonacci :: IO ()
printFibonacci = do
    putStrLn "How many fibonacci numbers do you want to see:"
    n <- (read <$> getLine) :: IO Int
    let initialState = [n,0,1]
    process initialState

process :: FibNums -> IO ()
process st = do
    let (finished, newSt) = runState addFib st
    if finished 
    then do
      putStrLn "Fibonacci numbers are:"
      print $ tail newSt
    else process newSt

addFib :: State FibNums Bool
addFib = do
    st <- get
    let newSt = st ++ [last st + last (init st)]
        finished = length (tail newSt) == head newSt 
    put newSt
    return finished

printFibonacci

First ask the user for a number and then run the **process** function that we provide with the initial state.

Out initial state is a list that hold the number of fibonacci elements we want and the first two fibonacci numbers.

In the **process** function we start the State monad **addFib** with the provided state and then check if the list is complete. If not we recursively call **process** again. 

The **addFib** State monad function first gets the state, then it updates the state and returns a Bool that inidicates if we computed the desired length.

## State monad examples

Let's now look at the previous fibonacci example where we rewrite the `addFib` state monad such that we use the `modify` and `gets` functions.

In [ ]:
addFib :: State FibNums Bool
addFib = do
    let updateSt st = st ++ [last st + last (init st)]
        isFinished st = length (tail st) == head st
    modify updateSt
    gets isFinished

printFibonacci

If we use this version with the previous code it works the same. We see there is no need to use the `get` and `set` functions in this example.

Instead we define functions that take in a state and use them to update the current state and the return variable of the state monad.

Next we write a game that takes in a string radomly composed of the characters `a`, `b` and `c`.

The game works like this: 
- we start with the intial state where the count is 0 and the game is OFF
- the caracter `c` switches the state of the game from OFF to ON or vice versa
- the caracter `a` adds +1 to the count if the game is ON
- the caracter `b` adds -1 to the count if the game is ON 

In the end the final count of the game is printed. We use the state monad to implement this game and also the `evalState` function.

In [ ]:
import Control.Monad.State ( evalState, MonadState(put, get), State )

type GameValue = String
type GameState = (Bool, Int)

playGame :: String -> State GameState GameValue
playGame [] = do
    (_, score) <- get
    return $ "Final score is: " ++ show score
playGame (x : xs) = do
    (on, score) <- get
    case x of
        'a' | on -> put (on, score + 1)
        'b' | on -> put (on, score - 1)
        'c' -> put (not on, score)
        _ -> put (on, score)
    playGame xs

main :: IO ()
main = do
    let startState = (False, 0::Int)
    putStrLn $ evalState (playGame "acaaabcbbcbaa") startState

In our last example we look at our initial Tic-Tac-Toe example and how we can re-write it with the use of the State monad.

Just to have an overview we will write out all also the code that is duplicated form the initial example.

In [ ]:
import Control.Monad.State (MonadState(get, put), runState, State)
import System.Random (mkStdGen, Random(randomR), StdGen)

data Player = XPlayer | OPlayer deriving Eq
data Choice = Empty | X | O deriving Eq

data GameState = GameState
  { currentBoard :: [Choice]
  , currentPlayer :: Player
  , generator :: StdGen
  }

main :: IO ()
main = do
  let gen = mkStdGen 1
      initState = GameState
                  [Empty | boardInd <- [1..9]]
                  XPlayer
                  gen
  playGame initState

playGame :: GameState -> IO ()
playGame gs = do
  let (gameFinished, newGS) = runState resolveTurn gs
  if gameFinished 
  then do
    putStrLn "Game results:"
    printBoard newGS
  else playGame newGS

resolveTurn :: State GameState Bool
resolveTurn = do
  choice <- chooseRandomMove
  applyMove choice
  isGameDone

chooseRandomMove :: State GameState Int
chooseRandomMove = do
  gs <- get
  let board = currentBoard gs
      openSpots = [ind | ind <- [0..8], board !! ind == Empty]
      gen = generator gs
  let (i, gen') = randomR (0, length openSpots - 1) gen
  put $ gs { generator = gen' }
  return $ (openSpots !! i) + 1

applyMove :: Int -> State GameState ()
applyMove choice = do
  gs <- get
  let player = currentPlayer gs
      board = currentBoard gs
      newBoard = if player == XPlayer
                 then [if ind /= choice then board !! (ind-1) else X | ind <- [1..9]]
                 else [if ind /= choice then board !! (ind-1) else O | ind <- [1..9]]
  put $ gs { currentPlayer = nextPlayer player, currentBoard = newBoard }

nextPlayer :: Player -> Player
nextPlayer XPlayer = OPlayer
nextPlayer OPlayer = XPlayer

isGameDone :: State GameState Bool
isGameDone = do
  gs <- get
  let board = currentBoard gs
      openSpots = [ind | ind <- [0..8], board !! ind == Empty]
  return $ length openSpots == 0

printBoard :: GameState -> IO ()
printBoard gs = do
    let board = currentBoard gs
    let stateToString st = case st of
                             Empty -> "-"
                             X -> "X"
                             O -> "O"
        printInd ind = stateToString $ board !! ind
    mapPutStr [printInd 0,"|", printInd 1,"|", printInd 2, "\n"]
    putStrLn "-----"
    mapPutStr [printInd 3,"|", printInd 4,"|", printInd 5, "\n"]
    putStrLn "-----"
    mapPutStr [printInd 6,"|", printInd 7,"|", printInd 8, "\n"]

mapPutStr :: [String] -> IO ()
mapPutStr [] = return ()
mapPutStr [x] = putStr x
mapPutStr (x:xs) = putStr x >> mapPutStr xs

main

Here we use the same **GameState** variable as before that now represents the state of our State monad.

In the **playGame** function we start the **resolveTurn** State monad and provide it the initial state.

This monad calls then three other State monads that update the state accordingly and the final return parameter tells us weather the game is finished.

## Recap

In this lesson we've discussed:

- the motivation for introducing the State monad type 

- the definition of the State monad type

- helper functions that work with the State monad

- examples that use the State monad type and functions that work with it